### Non-detections are common in transient astronomy, but sadly (sometimes for good reason) there is no single standard in how they are defined. This makes it tricky to write a single method to dealing with non-detections in a package like redback. 

### Instead, in this notebook we show one general way you can deal with non-detections in redback. And show you "how to fish", so you can also incorporate your own methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import redback

One of the simplest non-detections you can consider is that you simply know the flux/luminosity *whatever* does not exceed a certain value. Maybe this is because of physics e.g., maybe you have an empirical model for a certain transient and you know its never as bright as another transient, or you read a paper which says source was below a flux (whatever units) of X but you are not provided any details on the noise.

In such situations the easiest thing you can do is set a constraint on your prior to never violate this 'constraint'. How to do this is already shown in the `non_detection_example.py`, so I wont go into more detail here. I generally recommend going down this route unless you know what you are doing, you understand how your data was generated (including the non-detections), i.e., you understand what is the proper likelihood or are making a concious decision to use the wrong likelihood.

The next case assumes you do understand the likelihood and relies on changing the default likelihood in redback. Let's first start by simulating some transient data, including non-detections. I will use the `simulation` module to do this.

In [ ]:
from redback.simulate_transients import SimulateOpticalTransient
np.random.seed(1234)
parameters = {}
# We also can place the transient on the sky by setting the ra and dec parameters.
# This will be randomly set from the pointing if not given.
parameters = redback.priors.get_priors(model='arnett').sample()
parameters['mej'] = 10
parameters['t0_mjd_transient'] = 60500
parameters['redshift'] = 0.32
parameters['t0'] = parameters['t0_mjd_transient']
parameters['temperature_floor'] = 1000
parameters['kappa'] = 0.34
parameters['vej'] = 2500
parameters['kappa_gamma'] = 1e-2
parameters['f_nickel'] = 0.05

parameters['ra'] = 1.2
parameters['dec'] = -0.8
print(parameters)
model_kwargs = {}
# We now simulate a kilonova using the SimulateOpticalTransient class.
# Now specifying a survey string, which will load the pointings table from the tables directory in redback.
# These tables will need to be downloaded from zenodo using the redback.utils if not already present.
# Please look at the documentation for more details.
sn_sim = SimulateOpticalTransient.simulate_transient_in_rubin(model='arnett',
                                                              survey='Rubin_10yr_baseline',
                                                              parameters=parameters, 
                                                              model_kwargs=model_kwargs,
                                                              end_transient_time=250., 
                                                              snr_threshold=3.)

Lets load this into a redback transient object and plot the data with the true model to see what it all looks like.

First lets extract all the necessary attributes. Note we could directly create a transient object via the from simulation class method but I am doing it manually here so you can see what you need to do for your own transient.

In [ ]:
data = sn_sim.observations
time = data['time (days)'].values
flux = data['flux(erg/cm2/s)'].values
flux_err = data['flux_error'].values
bands = data['band'].values

# Limiting flux. This is a 5-sigma limit in the simulation module. 
limits = data['flux_limit']
# This is an array of booleans, O for non-detections, 1 for detections. 
detections = data['detected']
detected_mask = detections == 1.0

# just for convenience
upper_limits = sn_sim.observations[sn_sim.observations['detected'] != 1.0]

# let's create a redback transient object including only the detections
sn = redback.transient.Supernova(name='upperlimits', time=time[detected_mask], 
                                flux=flux[detected_mask], flux_err=flux_err[detected_mask], 
                                data_mode='flux', bands=bands[detected_mask])
# Make a dictionary for colors on the plot
band_colors = {'lsstg':'#4daf4a', 'lsstu':'#377eb8', 'lsstr':'#e41a1c', 
               'lsstz':'#a65628', 'lssti':'#ff7f00', 'lssty':'#984ea3'}
ax = sn.plot_data(band_colors=band_colors, show=False)
ax.set_ylim(1e-17, 4e-14)

# Let's plot the non-detections on top 
for band in band_colors.keys():
    up = upper_limits[upper_limits['band'] == band]
    ax.scatter(up['time (days)'], up['flux_limit'], s=100, 
                marker=r'$\downarrow$', color=band_colors[band])
    
    
# We can also plot the true data 
tt = np.linspace(0.1, 300, 200)
# specify output_format 
parameters['output_format'] = 'flux'
for band in band_colors.keys():
    parameters['bands'] = band
    out = redback.transient_models.supernova_models.arnett(tt, **parameters)
    ax.semilogy(tt, out, color=band_colors[band], label=band)

So you can see that the simulated transient has a bunch of detections and some non-detections in lsstu band and in lssty band. 

### Now what's the correct likelihood? 


Assuming the data is generated following model output + normal with zero mean and $\sigma \sim {\rm flux\_err}$ then the likelihood is Gaussian. If the limits are a $3\sigma$ threshold, then the 'sigma' of the upper limit is $\sim \rm{limit}/3$, and the likelihood is the CDF at obs_upperlimit - model / upper_limit_sigma

There is already a likelihood defined in redback to deal with such scenarios. Let's first show how it functions. Note this is an example you could use yourself, but ideally, after this tutorial, you also know how you would go about writing your own likelihood if you want to deal with a non-detection differently; for example, if your data is not in fact, Gaussian or reasonably approximated as a Gaussian e.g., when you have to count photons on one hand...

In [ ]:
redback.likelihoods.GaussianLikelihoodWithUpperLimits??

### Let's now use this likelihood. 

Now, the main difference between this and other redback likelihoods is that this needs the x/y/etc attributes to contain all the detections and non-detections alongside an extra array which flags which data points are upperlimits. So we can't just use the transient object populated x/y/etc

In [ ]:
function = redback.transient_models.supernova_models.arnett
x = time
y = np.zeros(len(time))
y[detected_mask] = flux[detected_mask]
y[~detected_mask] = limits[~detected_mask]
y_err = flux_err
kwargs = {"bands":bands, 'output_format':'flux'}
likelihood = redback.likelihoods.GaussianLikelihoodWithUpperLimits(x=x, y=y,sigma=y_err, 
                                                                   function=function, 
                                                                   detections=detections, 
                                                                   kwargs=kwargs, upper_limit_sigma=5.)

In [ ]:
# let's get a summary of the likelihood to make sure things look right.
likelihood.summary()

We can now sample with this likelihood as we normally would in redback

In [ ]:
priors = redback.priors.get_priors('arnett')
priors['redshift'] = parameters['redshift']
priors['kappa'] = parameters['kappa']
priors['kappa_gamma'] = parameters['kappa_gamma']
priors['vej'].maximum = 8000
injection_parameters = {}
for key in priors:
    injection_parameters[key] = parameters[key]
    
result = redback.fit_model(transient=sn, likelihood=likelihood, model='arnett', prior=priors, 
                          sampler='pymultinest', nlive=500, clean=True, plot=False, 
                           injection_parameters=parameters, model_kwargs=kwargs)

### Let's also do a detection only fit to see how it compares.

In [ ]:
# let's create a redback transient object including only the detections
sn = redback.transient.Supernova(name='only_detections', time=time[detected_mask], 
                                flux=flux[detected_mask], flux_err=flux_err[detected_mask], 
                                data_mode='flux', bands=bands[detected_mask])

priors = redback.priors.get_priors('arnett')
priors['redshift'] = parameters['redshift']
priors['kappa'] = parameters['kappa']
priors['kappa_gamma'] = parameters['kappa_gamma']
priors['vej'].maximum = 8000
injection_parameters = {}
for key in priors:
    injection_parameters[key] = parameters[key]

kwargs = {"bands":bands[detected_mask], 'output_format':'flux'}
# now we do not specify the likelihood, which defaults to using a Gaussian likelihood
# and only using detections
result_dets = redback.fit_model(transient=sn, model='arnett', prior=priors, 
                          sampler='pymultinest', nlive=500, clean=True, plot=False, 
                           injection_parameters=parameters, model_kwargs=kwargs)

Let's compare the two outputs

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 6))
axes = axes.ravel()

ax = axes[0]
ax = result.plot_lightcurve(band_colors=band_colors, show=False, random_models=100, 
                            uncertainty_mode='credible_intervals', axes=ax, plot_max_likelihood=False)

# Let's plot the non-detections on top 
for band in band_colors.keys():
    up = upper_limits[upper_limits['band'] == band]
    ax.scatter(up['time (days)'], up['flux_limit'], s=100, 
                marker=r'$\downarrow$', color=band_colors[band])
ax.set_ylim(1e-17, 4e-14)   
    
ax = axes[1]

ax = result_dets.plot_lightcurve(band_colors=band_colors, show=False, random_models=100, 
                            uncertainty_mode='credible_intervals', axes=ax, plot_max_likelihood=False)
ax.set_ylim(1e-17, 4e-14)

# Let's plot the non-detections on top 
for band in band_colors.keys():
    up = upper_limits[upper_limits['band'] == band]
    ax.scatter(up['time (days)'], up['flux_limit'], s=100, 
                marker=r'$\downarrow$', color=band_colors[band])
    

This was obviously a very exaggerated example with a lot of non-detections but now you know how to include them through the likelihood method. Keep in mind that it is easy to use different likelihoods in redback; so if you have a different data generating process with non-detections you can write your own likelihood, to make life even simpler; you could subclass the likelihood described above and just change the log_likelihood method to follow the math of your data-generating process.